# Chronic Stress Data Evaluation

이 노트북은 학습된 마우스 얼굴 표정 분석 모델을 사용하여 새로운 **Chronic Stress** 데이터셋을 평가합니다.

## 1. 환경 설정 및 라이브러리 로드

In [ ]:
import os
import glob
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

# 모델 클래스 로드 (경로 설정)
import sys
sys.path.append(os.path.abspath('.'))
from mouse_facial_expressions.models.train_task1_baseline_model import DeepSet

# 장치 설정 (MPS or CPU)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


## 2. 학습된 모델 로드
가장 성능이 좋았던 **Fold 0 (정확도 100%)** 모델을 로드합니다.

In [2]:
# 체크포인트 경로 (가장 성능이 좋은 모델 선택)
checkpoint_path = "models/checkpoints/task1-fold0-epoch=0-val_acc=1.00-v2.ckpt"

if not os.path.exists(checkpoint_path):
    # v2가 없으면 v1 시도
    checkpoint_path = "models/checkpoints/task1-fold0-epoch=0-val_acc=1.00-v1.ckpt"
    
if not os.path.exists(checkpoint_path):
    print("Error: Checkpoint not found!")
else:
    print(f"Loading model from: {checkpoint_path}")
    
    # 모델 로드 시 필요한 설정값 (학습 시 사용된 값과 동일하게 설정)
    # DeepSet.__init__에 필요한 인자들입니다.
    dummy_config = {
        'learning_rate': 0.01,
        'weight_decay': 0.0001,
        'momentum': 0.9,
        'label_smoothing': 0.1
    }
    # 클래스 가중치는 추론 시에는 loss 계산을 안 하므로 임의의 값으로 설정해도 무방합니다.
    dummy_class_weights = torch.tensor([1.0, 1.0], dtype=torch.float32)
    
    # 모델 로드
    model = DeepSet.load_from_checkpoint(
        checkpoint_path, 
        config=dummy_config, 
        class_weights=dummy_class_weights
    )
    model.to(device)
    model.eval()
    print("Model loaded successfully!")

Loading model from: models/checkpoints/task1-fold0-epoch=0-val_acc=1.00-v2.ckpt


Model loaded successfully!


## 3. 데이터셋 클래스 정의
추출된 프레임 이미지를 로드하기 위한 데이터셋 클래스를 정의합니다.

In [3]:
class InferenceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # 모든 png 파일 검색 (하위 폴더 포함)
        self.image_paths = glob.glob(os.path.join(root_dir, "**", "*.png"), recursive=True)
        print(f"Found {len(self.image_paths)} images.")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, img_path
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            return None, img_path

# 전처리 정의 (학습 데이터와 동일하게 300x300 리사이즈)
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
])

# 데이터셋 생성
data_dir = "data/chronic_stress_frames"
dataset = InferenceDataset(data_dir, transform=transform)

# 데이터로더 생성 (num_workers=0 for Jupyter compatibility)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=0)

Found 59768 images.


## 4. 추론 (Inference) 실행
데이터셋에 대해 예측을 수행합니다.

In [4]:
all_preds = []
all_probs = []
all_paths = []

print("Starting inference...")
with torch.no_grad():
    for i, (images, paths) in enumerate(tqdm(dataloader)):
        if images is None: continue
        
        images = images.to(device)
        
        # DeepSet 모델은 (Batch, Set, Channels, Height, Width) 형태의 5D 입력을 기대합니다.
        # 현재 images는 (Batch, Channels, Height, Width) 형태이므로,
        # Set 차원을 추가해줍니다 (Set size = 1).
        images = images.unsqueeze(1)  # (B, C, H, W) -> (B, 1, C, H, W)
        
        outputs = model(images)
        
        # 확률 계산 (Softmax)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())
        all_paths.extend(paths)

print("Inference complete!")

Starting inference...



  0%|          | 0/934 [00:00<?, ?it/s]


  0%|          | 1/934 [00:03<48:38,  3.13s/it]


  0%|          | 2/934 [00:06<48:07,  3.10s/it]


  0%|          | 3/934 [00:09<48:05,  3.10s/it]

...


100%|█████████▉| 932/934 [47:59<00:06,  3.13s/it]


100%|█████████▉| 933/934 [48:02<00:03,  3.13s/it]


100%|██████████| 934/934 [48:05<00:00,  3.03s/it]


100%|██████████| 934/934 [48:05<00:00,  3.09s/it]

Inference complete!


## 5. 결과 정리 및 저장
결과를 DataFrame으로 정리하고 CSV로 저장합니다.

In [5]:
# 결과 데이터프레임 생성
results_df = pd.DataFrame({
    'image_path': all_paths,
    'predicted_label': all_preds,
    'prob_0': [p[0] for p in all_probs],
    'prob_1': [p[1] for p in all_probs],
    # 'prob_2': [p[2] for p in all_probs], # 모델 클래스 수 확인 필요 (현재 2개로 보임)
    # 'prob_3': [p[3] for p in all_probs]
})

# 모델 클래스 수가 2개인지 4개인지 확인 필요. 
# DeepSet 클래스 정의에서 n_classes = 2로 되어있음.
# 만약 4개라면 위 코드를 수정해야 함.

# 비디오 이름 추출
results_df['video_name'] = results_df['image_path'].apply(lambda x: os.path.basename(os.path.dirname(x)))
results_df['frame_name'] = results_df['image_path'].apply(lambda x: os.path.basename(x))

# CSV 저장
output_csv = "chronic_stress_predictions.csv"
results_df.to_csv(output_csv, index=False)
print(f"Results saved to {output_csv}")

# 상위 5개 출력
results_df.head()

Results saved to chronic_stress_predictions.csv


,image_path,predicted_label,prob_0,prob_1,video_name,frame_name
0,data/chronic_stress_frames/Cohort5_22_FacialEx...,0,0.936968,0.063032,Cohort5_22_FacialExp,Cohort5_22_FacialExp_frame06900.png
1,data/chronic_stress_frames/Cohort5_22_FacialEx...,0,0.775582,0.224418,Cohort5_22_FacialExp,Cohort5_22_FacialExp_frame10810.png
2,data/chronic_stress_frames/Cohort5_22_FacialEx...,0,0.885293,0.114707,Cohort5_22_FacialExp,Cohort5_22_FacialExp_frame15940.png
3,data/chronic_stress_frames/Cohort5_22_FacialEx...,0,0.798348,0.201652,Cohort5_22_FacialExp,Cohort5_22_FacialExp_frame03850.png
4,data/chronic_stress_frames/Cohort5_22_FacialEx...,0,0.912282,0.087718,Cohort5_22_FacialExp,Cohort5_22_FacialExp_frame08050.png


## 6. 비디오별 결과 집계
각 비디오별로 어떤 표정이 가장 많이 나타났는지 확인합니다.

In [6]:
# 비디오별 예측 빈도 집계
video_summary = results_df.groupby(['video_name', 'predicted_label']).size().unstack(fill_value=0)

# 비율로 변환
video_summary_pct = video_summary.div(video_summary.sum(axis=1), axis=0) * 100

print("Video-level Summary (Count):")
display(video_summary)

print("\nVideo-level Summary (Percentage):")
display(video_summary_pct)

# 저장
video_summary.to_csv("chronic_stress_video_summary.csv")

Video-level Summary (Count):


predicted_label,0,1
video_name,,
Cohort 5_12_FacialExp,1648,0
Cohort1 _26_facial exp,1598,0
Cohort1_19_FacialExp,779,752
Cohort1_20_FacialExp,1429,93
Cohort1_3_FacialExp,1462,63
Cohort1_40_FacialExp,1205,301
Cohort1_41_FacialExp,1161,0
Cohort1_45_FacialExp.,1524,0
Cohort1_5_FacialExp,1504,1



Video-level Summary (Percentage):


predicted_label,0,1
video_name,,
Cohort 5_12_FacialExp,100.000000,0.000000
Cohort1 _26_facial exp,100.000000,0.000000
Cohort1_19_FacialExp,50.881777,49.118223
Cohort1_20_FacialExp,93.889619,6.110381
Cohort1_3_FacialExp,95.868852,4.131148
Cohort1_40_FacialExp,80.013280,19.986720
Cohort1_41_FacialExp,100.000000,0.000000
Cohort1_45_FacialExp.,100.000000,0.000000
Cohort1_5_FacialExp,99.933555,0.066445
